In [1]:
1+1

2

In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain.agents import initialize_agent, Tool, AgentType

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


def add(x: str, y:str) -> str:
    print(f"x:{x}, y:{y}")
    try:
        return str(float(x) + float(y))
    except Exception:
        return ValueError("계산 실패")

def lookup_population(city: str) -> str:
    print(f"city:{city}")

    population = {
        "서울": "900만",
        "부산": "300만",
    }
    return population.get(city.strip(),"정보 없음")


def my_name(name: str) -> str:
    print(f"name:{name}")

    population = {
        "태양": "100억",
        "주영": "300만",
        "영현": "100억",
    }
    return population.get(name.strip(), f"{name}정보 없음")


tools = [
    Tool(
        name="Add",
        func=lambda x: add(*x.split()),
        description="두 숫자를 더합니다 (예: 5 2)",
    ),
    Tool(
        name="PopulationLookup",
        func=lookup_population,
        description="도시명으로 인구를 알려줍니다 (예: 서울)",
    ),
    Tool(
        name="MyName",
        func=my_name,
        description="이름으로 가지고 있는 돈을 알려준다. (예: 영현)",
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    verbose=True,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

print("ReAct 실행:")
# print(agent.run("5에 2를 더하고, 서울 인구도 알려줘, 그리고 태양이 얼마가지고 있는지도 알려줘."))
print(agent.run("5에 2를 더하지 말고 빼, 서울 인구도 알려줘, 그리고 태양이 얼마가지고 있는지도 알려줘."))

ReAct 실행:


> Entering new AgentExecutor chain...
I need to perform three actions: first, add 5 and 2; second, look up the population of Seoul; and third, find out how much money 태양 has. 

Action: Add  
Action Input: 5 2  x:5, y:2

Observation: 7.0
Thought:I have the result of adding 5 and 2, which is 7. Now I will look up the population of Seoul.

Action: PopulationLookup  
Action Input: 서울  city:서울

Observation: 900만
Thought:I have the population of Seoul, which is 900만. Now I will find out how much money 태양 has.

Action: MyName  
Action Input: 태양  name:태양

Observation: 100억
Thought:I now know the final answer. 
Final Answer: 5에 2를 더하면 7이고, 서울의 인구는 900만이며, 태양은 100억을 가지고 있습니다.

> Finished chain.
5에 2를 더하면 7이고, 서울의 인구는 900만이며, 태양은 100억을 가지고 있습니다.
